In [1]:
from pathlib import Path

import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

# 
import sys ; sys.path.insert(0, "..")
from src.datasets.dbg import dBGDataset
from src.models.gcn import GCN

In [2]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 4080'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
dbg_dataset = dBGDataset(root=Path("../data/datasets/subsample-G-random-6mer"), k=6, 
                         files=[
                             "../data/subsample-clade_G.fasta",
                             "../data/subsample-random.fasta",
                            #  "../data/subsample-clade_O.fasta",
                            #  "../data/subsample-clade_S.fasta",
                            ], 
                        format="fasta")

In [5]:
dbg_dataset.len()


9546

In [6]:
dbg_dataset.num_node_features

1

In [7]:
dbg_dataset.num_classes

2

In [8]:
model = GCN(hidden_channels=64, num_node_features=dbg_dataset.num_node_features, num_classes=2)
model.to(device)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
  (softmax): Softmax(dim=-1)
)


In [17]:
import numpy as np 

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

dbg_dataset.shuffle()
train_loader = DataLoader(dbg_dataset, batch_size = 32 , shuffle = True, num_workers=4)
# test_loader = DataLoader(dbg_dataset[0.8* dbg_dataset.len():], batch_size = 2 , shuffle = False, num_workers=2)


In [16]:
for epoch in range(50):
    lloss = []
    for data in train_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        # out = F.softmax(out, dim=-1)
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        # for name, param in model.named_parameters():
        #     print(name, param.grad)
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        
        lloss.append(loss.cpu().detach().numpy())

    m_loss = np.array(lloss).mean()

    lloss.append(loss.cpu().detach().numpy())
    print(f'Epoch: {epoch:03d}, Loss: {m_loss:.4f}') 


Epoch: 000, Loss: 0.6923
Epoch: 001, Loss: 0.6930
Epoch: 002, Loss: 0.6935
Epoch: 003, Loss: 0.6930
Epoch: 004, Loss: 0.6927
Epoch: 005, Loss: 0.6924
Epoch: 006, Loss: 0.6924
Epoch: 007, Loss: 0.6924
Epoch: 008, Loss: 0.6930
Epoch: 009, Loss: 0.6925
Epoch: 010, Loss: 0.6923
Epoch: 011, Loss: 0.6926
Epoch: 012, Loss: 0.6924
Epoch: 013, Loss: 0.6927
Epoch: 014, Loss: 0.6927
Epoch: 015, Loss: 0.6931
Epoch: 016, Loss: 0.6925
Epoch: 017, Loss: 0.6920
Epoch: 018, Loss: 0.6926
Epoch: 019, Loss: 0.6924
Epoch: 020, Loss: 0.6924
Epoch: 021, Loss: 0.6924
Epoch: 022, Loss: 0.6925
Epoch: 023, Loss: 0.6925
Epoch: 024, Loss: 0.6934
Epoch: 025, Loss: 0.6927
Epoch: 026, Loss: 0.6926
Epoch: 027, Loss: 0.6925
Epoch: 028, Loss: 0.6924
Epoch: 029, Loss: 0.6930


KeyboardInterrupt: 

In [12]:
model(data.x, data.edge_index, data.batch)

tensor([[0.4045, 0.5955],
        [0.4045, 0.5955]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [13]:
data.y

tensor([1, 1], device='cuda:0')